# Fully Convolutional Network - Semantic Segmentation

![image.png](imgs/2.png)

![image.png](imgs/3.png)

In [ ]:
import os
import os.path as osp
import pytz
import torch

import warnings
warnings.filterwarnings('ignore')

configurations = {
    # same configuration as original work
    # https://github.com/shelhamer/fcn.berkeleyvision.org
    1: dict(
        max_iteration=100000,
        lr=1.0e-10,
        momentum=0.99,
        weight_decay=0.0005,
        interval_validate=4000,
    )
}

In [ ]:
from types import SimpleNamespace
opts = SimpleNamespace()
opts.cfg = configurations[1]
opts.resume = ''
print(opts.cfg)

In [ ]:
from utils import get_log_dir
opts.out = get_log_dir('resnet101', 1, opts.cfg)
print(opts.out)

In [ ]:
gpu = 1
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
cuda = torch.cuda.is_available()
print('Cuda: {}'.format(cuda))
opts.cuda = 'cuda' if cuda else 'cpu'
opts.mode = 'train'
opts.backbone = 'resnet'
opts.fcn = '101'

## PascalVOC Dataset - Downloaded on _`root`_ variable

In [ ]:
root = './data/Pascal_VOC'
print(root)

In [ ]:
from data_loader import Pascal_Data
kwargs = {'num_workers': 4} if cuda else {}
train_loader = torch.utils.data.DataLoader(
        Pascal_Data(root, image_set='train', backbone=opts.backbone),
        batch_size=1, shuffle=True, **kwargs)
val_loader = torch.utils.data.DataLoader(
        Pascal_Data(root, image_set='val', backbone=opts.backbone),
        batch_size=1, shuffle=False, **kwargs)
data_loader = [train_loader, val_loader]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
for data, target in train_loader: break
print(data.shape)
print(target.shape)
data.min()
data_show, label_show = train_loader.dataset.untransform(data[0].cpu().clone(), target[0].cpu().clone())

plt.imshow(data_show)
plt.show()

def imshow_label(label_show):
    import matplotlib
    import numpy as np
    cmap = plt.cm.jet
    # extract all colors from the .jet map
    cmaplist = [cmap(i) for i in range(cmap.N)]
    cmaplist[0] = (0.0,0.0,0.0,1.0)
    cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)
    # define the bins and normalize
    bounds = np.arange(0,len(train_loader.dataset.class_names))
    norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
    plt.imshow(label_show, cmap=cmap, norm=norm)
    cbar = plt.colorbar(ticks=bounds)
    cbar.ax.set_yticklabels(train_loader.dataset.class_names)
    plt.show()    
    
imshow_label(label_show)


## FCN - Model

In [ ]:
import numpy as np
import torch.nn as nn
from torchvision.models.segmentation import fcn_resnet101

class FCN(nn.Module):
    def __init__(self, n_class=21):

        super(FCN, self).__init__()
        self.fcn = fcn_resnet101(pretrained=False, num_classes=21)
        # Uses bilinear interpolation for upsampling
        # https://github.com/pytorch/vision/blob/master/torchvision/models/segmentation/_utils.py

    def forward(self, x, debug=False):
        return self.fcn(x)['out']          

In [ ]:
model = FCN(n_class=21)
model.to(opts.cuda)

In [ ]:
iter_loader=iter(train_loader)
data, target = next(iter_loader)
data = data.to(opts.cuda)
with torch.no_grad():
    output = model(data)

In [ ]:
print('input: ', data.shape)
print('output: ', output.data.shape)

In [ ]:
data, target = next(iter_loader)
data = data.to(opts.cuda)
with torch.no_grad():
    print('input: ', data.shape)
    output = model.fcn.backbone(data)['out']
    print('backbone: ', output.data.shape)
    output = model.fcn.classifier(output)
    print('output: ', output.data.shape)

In [ ]:
data, target = next(iter_loader)
data = data.to(opts.cuda)
with torch.no_grad():
    print('input: ', data.shape)
    output = model.fcn.backbone(data)['out']
    print('backbone: ', output.data.shape)
    output = model.fcn.classifier(output)
    print('output: ', output.data.shape)

In [ ]:
%matplotlib inline
from trainer import Trainer

In [ ]:
trainer = Trainer(data_loader, opts)

In [ ]:
print(opts.cfg.get('interval_validate', len(train_loader))) #Validate every 4000 iterations
print(opts.out)

In [ ]:
start_epoch = 0
start_iteration = 0
if opts.resume:
    start_epoch = checkpoint['epoch']
    start_iteration = checkpoint['iteration']

In [ ]:
trainer.epoch = start_epoch
trainer.iteration = start_iteration
trainer.Train()